In [1]:
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("hf_models/query_classify_v2")
model_ort = ORTModelForSeq2SeqLM.from_pretrained("hf_models/query_classify_v2", from_transformers=False)

inputs = tokenizer("My name is Eustache and I like to", return_tensors="pt")

gen_tokens = model_ort.generate(**inputs)
outputs = tokenizer.batch_decode(gen_tokens)

/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [2]:
outputs

['<pad> jewelry & accessories > customized jewelry > c् pendants</s>']

In [3]:
%%timeit
gen_tokens = model_ort.generate(**inputs)

/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


457 ms ± 10.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
